# Phase 1: Create a Word-Embedding layer 

In [1]:
#imports
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
import networkx as nx
import torch

In [2]:
def read_data(datafile):
    with open(datafile) as f:
        lines = f.readlines()
    return lines

In [3]:
glove_embedding = WordEmbeddings('glove')
words = read_data('../data/vertomul.txt')
words = [line.strip() for line in open('../data/vertomul.txt')]

In [4]:
word_emb = {}
for word in words:
    sentence = Sentence(word) # --> strip() removes the white space from beginning and end of word
     # embed a sentence using glove.
    glove_embedding.embed(sentence)
    for token in sentence:

        word_emb[word]=token.embedding
    

###  We decided to compute the distance using cosine similarity

In [5]:
G = nx.Graph()

In [6]:
G.add_nodes_from(words)

In [7]:
len(list(G.nodes))

529

A similarity of 1 means that the words are the same.
Cosine similarity value smaller than 80 makes the edges increase by a lot. 

In [9]:
for item in word_emb:
    x = word_emb[item]
    
    for word in word_emb:
        if word is not item:
            y = word_emb[word]
            cosine_sim = torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))
            
            if cosine_sim > .8:
                G.add_edge(item, word)
#             print("x:", item, "y:", word, "distance: ", cosine_sim)

In [11]:
G.number_of_nodes()

529

In [12]:
G.number_of_edges()

246

The graph is not connected 

In [15]:
print(nx.is_connected(G))

False


Nodes with higher closeness centrality are better connected with the rest of the nodes. I am assuming that nodes with a closeness centrality equal to zero are not connected to any other node. 

In [35]:
cc = nx.closeness_centrality(G)